In [14]:
from random import shuffle 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import movie_reviews 

%matplotlib inline

np.seterr(all='raise')

# Download if you don't have already!
# nltk.download('movie_reviews')

{'divide': 'raise', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

In [15]:
# Review categories
print (movie_reviews.categories()) # Output: [u'neg', u'pos']
 
# Total positive reviews
print (len(movie_reviews.fileids('pos'))) # Output: 1000
 
# Total negative reviews
print (len(movie_reviews.fileids('neg'))) # Output: 1000

documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        #documents.append((list(movie_reviews.words(fileid)), category))
        documents.append((movie_reviews.words(fileid), category))
shuffle(documents)

['neg', 'pos']
1000
1000


In [23]:
for i in range(10):
    print(documents[i][1], ":", " ".join(documents[i][0])[:100], "...")

pos : jack nicholson has a funny way of playing characters with very few redeeming qualities , but whom yo ...
neg : capsule : this super - light situation comedy from sweden tells the story of two close friends with  ...
pos : blade is the movie that shows that wesley snipes really can live up to his potential as one of holly ...
pos : in recent years , harrison ford has been such a grave screen presence , scowling through the likes o ...
pos : billy bob thornton , who had a sudden rise to fame with 1996 ' s sling blade after spending years as ...
neg : it ' s actually not so bad that dreamworks decided to release the love letter at about the same time ...
pos : just in time for halloween and christmas , the reissue of the nightmare before christmas couldn ' t  ...
pos : with the abundance of trite , recycled movies in the late ' 90s , there is a tremendous demand by mo ...
neg : a disappointing biography about the homosexual relationship of two famous 19th century french poets  ...
p